In [ ]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
from PIL import Image
import time
import matplotlib.pyplot as plt

def pointer_to_numpy(ptr,length=0):
    if length ==0:
        length = len(ptr)
    return np.array([ptr[xx] for xx in range(length)])


def get_tile(tile_id):
    tile = []
    for yy in range(8):
        row = []
        lb = bg_ram[tile_id*16+yy]
        ub = bg_ram[tile_id*16+yy+8]
        for xx in range(8):
            l = lb&1
            u = ub&1
            v = l+u*2
            lb >>= 1
            ub >>= 1
            row.append(v)
        tile.append(list(reversed(row)))
    tile = np.array(tile)
    return tile

In [ ]:
def hold(mask, duration):
    return [mask for i in range(duration)]



imgBuffer = VectorBytes()
def outputImage(emu, name, buf=imgBuffer):
    emu.imageInto(buf)
    outImg = Image.frombytes("RGBA", (256, 240), str(bytearray(buf)))
    outImg.save(name + ".png")
    
    
def sprite_attributes_to_dict(attributes):
    masks = {'background' : 0x20,
        'hflip' : 0x40,
        'vflip'  : 0x80}
    sprite_attributes = {mask:attributes&masks[mask]>0 for mask in masks}
    sprite_attributes['palette'] = attributes & 0x03
    return sprite_attributes

In [ ]:

inputVec = fceulib.readInputs('videos/algar-batman.fm2')
emu = fceulib.runGame('batman.nes')
startInputs = hold(0x0, 120) + hold(START | JUMP, 30) + hold(0x0, 150)+ hold(START | JUMP, 30) #+ hold(0x0, 1000)
jumpInputs = [hold(0x0, 5) + hold(JUMP, t) + hold(0x0, 300)
              for t in range(5, 300)]
#st = time.clock()
#for m in startInputs:
#    emu.step(m, 0x0)
#et = time.clock()
#print 'speedup',float(len(startInputs))/(60.*(et-st))
#emu.stepFull(0x0, 0x0)
start = VectorBytes()
#print("SAVE")
emu.save(start)


fg_pat_addr = 0x0000 if (emu.fc.ppu.values[0] & (1 << 4)) else 0x1000;
fg_ram = pointer_to_numpy(emu.fc.cart.getVPageChunk(fg_pat_addr),0x1000)
bg_pat_addr = 0x1000 if (emu.fc.ppu.values[0] & (1 << 4)) else 0x0000;
bg_ram = pointer_to_numpy(emu.fc.cart.getVPageChunk(bg_pat_addr),0x1000)


def get_sprite(tile_id):
    tile = []
    for yy in range(8):
        row = []
        lb = fg_ram[tile_id*16+yy]
        ub = fg_ram[tile_id*16+yy+8]
        for xx in range(8):
            l = lb&1
            u = ub&1
            v = l+u*2
            lb >>= 1
            ub >>= 1
            row.append(v)
        tile.append(list(reversed(row)))
    tile = np.array(tile)
    return tile

In [ ]:
peekevery = 1000
emu.load(start)
image_prefix = 'batman'
for timestep, inp in enumerate(inputVec):
    emu.stepFull(inp,0x0)
    if not (timestep % 300 == 0):
        continue
    print timestep
    nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
    ram_ptr = emu.fc.fceu.RAM
    nt = nt[:960].reshape(30,32)
    outputImage(emu, '{}_{}'.format(image_prefix,timestep))
    pt = pointer_to_numpy(emu.fc.ppu.PALRAM)
    sprite_ram  = pointer_to_numpy(emu.fc.ppu.SPRAM)
    
    to_plot = []
    for ii in range(0,len(sprite_ram),4):
        if sprite_ram[ii] < 240:
            print ii/4,'x: {}, y: {}, id:{}, attributes:{},{}'.format(sprite_ram[ii+3],
                                                          sprite_ram[ii]+1,
                                                          sprite_ram[ii+1],
                                                          sprite_attributes_to_dict(sprite_ram[ii+2]),
                                                          sprite_ram[ii+2])
            if sprite_ram[ii+1] > 0:
                to_plot.append([sprite_ram[ii+3]*1.,sprite_ram[ii]*1.+1,ii/4])
    if len(to_plot) > 0:
        to_plot = np.array(to_plot,dtype='float')
        print to_plot
        plt.plot(to_plot[:,0]/8.,to_plot[:,1]/8.,'rx')
        
    ram_ptr = pointer_to_numpy(emu.fc.fceu.RAM)
    for ram_index,d in enumerate(ram_ptr):
        for sprite_index,sprite_pos in enumerate(to_plot):
            if d == sprite_pos[0]:
                print ram_index , 'possibly',sprite_index,',',sprite_pos[2],'x = ',d
            if d == sprite_pos[1]-1:
                print ram_index, 'possibly', sprite_index,',',sprite_pos[2],'y = ',d
                
    plt.imshow(nt,interpolation='none')
    plt.show()

In [ ]:

#plt.imshow(nt,interpolation='none')
#plt.show()

In [ ]:

for ii in range(1):
    print ii
    plt.imshow(get_sprite(ii
                     ),interpolation='nearest', cmap=plt.get_cmap('bone'))
    plt.show()
#  const uint8 *vram = &emu->GetFC()->cart->
# VPage[bg_pat_addr >> 10][bg_pat_addr];

In [ ]:

plt.imshow(get_tile(56),interpolation='nearest')
plt.show()
#  const uint8 *vram = &emu->GetFC()->cart->
# VPage[bg_pat_addr >> 10][bg_pat_addr];

In [ ]:
help(fceulib)